In [ ]:
# !pip install tweepy

In [ ]:
# import tweepy
# import configparser
# import pandas as pd


In [ ]:

# # read configs
# # config = configparser.ConfigParser()
# # config.read('config.ini')

# api_key = 'sZ41ftVMY7zqoSVdcp4fpodZz'
# api_key_secret = '1X043vSW8FV1hedSENFbgaaryENa8dh9A9LFg5sdCu7AuZMnX7'

# access_token = '1268928347494420480-rhplk9UBYdbWXz0KERZCsDlbJv0tLr'
# access_token_secret = 'EuY9IdIPxxL3038W7Cf3pMCbpXVWvq1R2CmcnddYRZcYa'

# # authentication
# auth = tweepy.OAuthHandler(api_key, api_key_secret)
# auth.set_access_token(access_token, access_token_secret)

# api = tweepy.API(auth)

# public_tweets = api.home_timeline()

# print(public_tweets)

# # create dataframe
# # columns = ['Time', 'User', 'Tweet']
# # data = []
# # for tweet in public_tweets:
# #     data.append([tweet.created_at, tweet.user.screen_name, tweet.text])

# # df = pd.DataFrame(data, columns=columns)

# # df.to_csv('tweets.csv')

In [ ]:
!pip install snscrape

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
zzz=[]
# query = "(from:elonmusk) until:2020-01-01 since:2010-01-01"

# query = "indian stocks"
# tweets = []
# limit = 5


# for tweet in sntwitter.TwitterSearchScraper(query).get_items():
#     # print(tweet.content)
#     # break
#     if len(tweets) == limit:
#         break
#     else:
#         tweets.append([tweet.date, tweet.content])
        
# df = pd.DataFrame(tweets, columns=['Date', 'Tweet'])
# print(df)

# to save to csv
# df.to_csv('tweets.csv')

In [ ]:
# df

In [ ]:

# for tweet in sntwitter.TwitterSearchScraper(query).get_items():
#     if len(tweets) == limit:
#         break
#     else:
#         tweets.append([tweet.content])


In [ ]:
# tweets

In [ ]:
# df

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)


# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)




In [ ]:
# def sentm(text):

#   # text = """@Curious_Indian_ But after selling we'll have to research again to buy new stocks, that is also time consuming"""
#   respp=[]
#   text = preprocess(text)
#   encoded_input = tokenizer(text, return_tensors='pt')
#   output = model(**encoded_input)
#   scores = output[0][0].detach().numpy()
#   scores = softmax(scores)

#   ranking = np.argsort(scores)
#   ranking = ranking[::-1]
#   for i in range(scores.shape[0]):
#       l = labels[ranking[i]]
#       s = scores[ranking[i]]
#       print(f"{i+1}) {l} {np.round(float(s), 4)}")
#       respp.append(np.round(float(s), 4))

#   # print(f"{0}) {labels[ranking[0]]} {np.round(float(scores[ranking[0]]), 4)}")
#   # resp=np.round(float(scores[ranking[0]]), 4)
#   return respp


# # # TF
# # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# # model.save_pretrained(MODEL)

# # text = "Good night 😊"
# # encoded_input = tokenizer(text, return_tensors='tf')
# # output = model(encoded_input)
# # scores = output[0][0].numpy()
# # scores = softmax(scores)



In [ ]:
def sentmm(text):
    # text = """you are a  boy"""
    respp=[]
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores

In [ ]:
scores[2]

In [ ]:
s= sentmm('''Curious_Indian_ But after selling we'll have to research again to buy new stocks, that is also time consuming''')

1) negative 0.5477
2) neutral 0.4279
3) positive 0.0244


In [ ]:
import statistics

In [ ]:
def tweett(query,limit):
  tweets = []
  tweetscore = []

  for tweet in sntwitter.TwitterSearchScraper(query).get_items():
      # print(tweet.content)
      # break
      if len(tweets) == limit:
          break
      else:
          tweets.append([tweet.date, tweet.content])
          x=sentmm(tweet.content)
          tweetscore.append([x[0],x[1],x[2]])

  sc = pd.DataFrame(tweetscore, columns=['-ve', '*','+ve'])
  df = pd.DataFrame(tweets, columns=['Date', 'Tweet'])
  # print(df)
  # mn=statistics.mean(tweetscore)

  return sc


In [ ]:
x=tweett('indian stocks today',50)
zzz.append([x["-ve"].mean(),x["*"].mean(),x["+ve"].mean()])
zzz

[[0.43886054, 0.46473092, 0.096408516], [0.06496478, 0.7541483, 0.18088686]]

In [ ]:
x=tweett('indian politics',50)
zzz.append([x["-ve"].mean(),x["*"].mean(),x["+ve"].mean()])
zzz

[[0.43886054, 0.46473092, 0.096408516]]